Classificação de Texto Multilabel


José Augusto de Almeida Neto


## Importar bibliotecas


In [2]:
import os
import time
import pandas as pd
import torch
from dotenv import load_dotenv
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer

c:\VSCODEProjects\artigo2024\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carregar datasets


In [14]:
# Carregar dataset
df_case_study = pd.read_csv('../datasets/dataset-case_study-tratado.csv')
df_case_study

,restaurante_name,user_name,review_date,stars,comment_text,year,platform,language,comment_text_translated
0,cipriani,Viva Gastronomia,9 anos atrás no Zomato,1,Hotelaria e gastronomia são duas artes demand...,2015,Zomato,pt,Hotelaria e gastronomia são duas artes demanda...
1,cipriani,Аннa Шкаленкова,11 dias atrás no Google,1,It was possible to order only a tasting menu;...,2024,Google,en,Foi possível encomendar apenas um menu de degu...
2,cipriani,Roberto Barretto,um mês atrás no Google,1,Grade 10! Food: 5 Service: 4 Atmosph...,Unknown,Google,en,Grade 10!
3,cipriani,GRUPO VANDERSON SOARES,um mês atrás no Google,1,Wonderful Food: 5 Service: 5 Atmosp...,Unknown,Google,en,comida maravilhosa: 5
4,cipriani,Maria Baptista,um mês atrás no Google,1,We had a great dining experience at Cipriani....,Unknown,Google,en,Tivemos uma ótima experiência gastronômica em ...
...,...,...,...,...,...,...,...,...,...
4968,tangara,Christopher Lopez,4 anos atrás no Google,1,Great place for lunch. Nice scenary good wine ...,2020,Google,en,Ótimo lugar para o almoço.Lista de vinhos de b...
4969,tangara,Silvio Marcos,4 anos atrás no Google,1,Boa comida. Sabor marcante,2020,Google,pt,Boa comida. Sabor marcante
4970,tangara,Maria Lins,4 anos atrás no Google,1,Comida divina,2020,Google,it,comida divina
4971,tangara,Ricardo Polisel Alves,5 anos atrás no Foursquare,0,Tasting Menu was indeed very good. Service wa...,2019,Foursquare,en,O menu de degustação foi realmente muito bom.O...


In [15]:
labels = ['ambiente', 'bebida', 'comida', 'geral',
          'localização', 'outros', 'preço', 'serviço']

In [17]:
# Renomeando a coluna de df
df = df_case_study[['comment_text_translated']].copy()
df.rename(columns={'comment_text_translated': 'sentenca'}, inplace=True)
for label in labels:
    df[label] = pd.NA 
df

,sentenca,ambiente,bebida,comida,geral,localização,outros,preço,serviço
0,Hotelaria e gastronomia são duas artes demanda...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Foi possível encomendar apenas um menu de degu...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Grade 10!,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,comida maravilhosa: 5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Tivemos uma ótima experiência gastronômica em ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
4968,Ótimo lugar para o almoço.Lista de vinhos de b...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4969,Boa comida. Sabor marcante,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4970,comida divina,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4971,O menu de degustação foi realmente muito bom.O...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Hiperparâmetros


In [18]:
# Hiperparâmetros
max_len = 128
test_batch_size = 4
num_labels = len(labels)
threshold = 0.28

## Funções auxiliares


In [22]:
class TokenizeDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['sentenca']
        self.targets = self.df[labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [23]:
def run_prediction(model, df_unlabeled):
    start_time = time.time()

    # Carregar o tokenizador BERT
    tokenizer = BertTokenizer.from_pretrained(
        'neuralmind/bert-base-portuguese-cased')

    # Tokenizar o dataset
    tokenized_unlabeled_data = TokenizeDataset(df_unlabeled, tokenizer, max_len)

    # Criar DataLoader para carregar os dados em lotes durante o treinamento
    unlabeled_loader = DataLoader(tokenized_unlabeled_data,
                              batch_size=test_batch_size, shuffle=True)

    model.eval()
    all_predictions = []
    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        for batch in unlabeled_loader:
            inputs_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']

            outputs = model(input_ids=inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            logits = outputs.logits
            predicted_probs = sigmoid(logits)

            predicted_labels = (predicted_probs > threshold).float()
            predicted_labels_np = predicted_labels.cpu().numpy()

            all_predictions.extend(predicted_labels_np)
    
    # Calcular o tempo total de teste
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)

    print(
        f"\nTempo de predição: {int(hours):02}:{int(minutes):02}:{seconds:.2f}")

    return all_predictions

## Aplicação do modelo

In [24]:
# Importar o modelo
model_dir = './bertimbau-model'
model = BertForSequenceClassification.from_pretrained(model_dir)

In [25]:
name = 'teste1'

# Fazer previsões no dataset sem anotações
predictions = run_prediction(model, df)

# Salvar as previsões em um arquivo CSV
predictions_df = pd.DataFrame(predictions, columns=labels)
predictions_df.to_csv(f'../datasets/dataset-case_study_predictions-{name}.csv', index=False)

c:\VSCODEProjects\artigo2024\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.